# Blobtools check

In [1]:
WKDIR=/workspace/hraijc/Bee/BeeT/Assemblies/blobplots

FLYE10=/workspace/hraijc/Bee/BeeT/Assemblies/Flye_BeeT_q10/Flye_BeeT_q10.fasta

ONTREADS_Q15=/workspace/hraijc/Bee/BeeT/BeeT_ONT14_1_basecalling/fastq/BeeT_Q15.fastq

ILLREAD1=/workspace/hraijc/Bee/input/Leioproctus_imitatus/popgen/NI-FR-Li-F-61-40567916_S16_L004_R1_001.clean.fastq
ILLREAD2=/workspace/hraijc/Bee/input/Leioproctus_imitatus/popgen/NI-FR-Li-F-61-40567916_S16_L004_R2_001.clean.fastq

cd ${WKDIR}

In [22]:
mkdir -p ${WKDIR}/log
mkdir -p ${WKDIR}/blasthits
mkdir -p ${WKDIR}/bamfiles



## Blobtools for contam check.

In [28]:
# Create coverage files.
sbatch << EOF
#!/bin/bash
#SBATCH -J minimap_samtools
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=32
#SBATCH --mem=20G
#SBATCH --time=11:00:00

module load minimap2/2.22
module load samtools/1.16

## Map ##
#minimap2 -t 32 -ax map-ont ${FLYE10} ${ONTREADS_Q15} > ${WKDIR}/bamfiles/BeeT_FLYE10_ONT15.sam

## Sam -> bam ##
#samtools view --threads 32 -b ${WKDIR}/bamfiles/BeeT_FLYE10_ONT15.sam > ${WKDIR}/bamfiles/BeeT_FLYE10_ONT15.bam

## Sort bam ##
samtools sort --threads 32 ${WKDIR}/bamfiles/BeeT_FLYE10_ONT15.bam > ${WKDIR}/bamfiles/BeeT_FLYE10_ONT15.sorted.bam

## index bam ##
samtools index ${WKDIR}/bamfiles/BeeT_FLYE10_ONT15.sorted.bam


EOF


Submitted batch job 1404229


In [3]:
# Coverage file of pop-gen data 
# Hopefully this has less microbial contamination than the gut sample.
ILLREAD1=/workspace/hraijc/Bee/input/Leioproctus_imitatus/popgen/NI-FR-Li-F-61-40567916_S16_L004_R1_001.clean.fastq
ILLREAD2=/workspace/hraijc/Bee/input/Leioproctus_imitatus/popgen/NI-FR-Li-F-61-40567916_S16_L004_R2_001.clean.fastq

sbatch << EOF
#!/bin/bash
#SBATCH -J minimap_samtools
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=16
#SBATCH --mem=10G
#SBATCH --time=2:00:00

module load minimap2/2.22
module load samtools/1.16

## Map ##
minimap2 -t 16 -ax sr ${FLYE10} ${ILLREAD1} ${ILLREAD2} > ${WKDIR}/bamfiles/BeeT_FLYE10_NIFRLiF61.sam

## Sam -> bam ##
samtools view --threads 32 -b ${WKDIR}/bamfiles/BeeT_FLYE10_NIFRLiF61.sam > ${WKDIR}/bamfiles/BeeT_FLYE10_NIFRLiF61.bam

## Sort bam ##
samtools sort --threads 32 ${WKDIR}/bamfiles/BeeT_FLYE10_NIFRLiF61.bam > ${WKDIR}/bamfiles/BeeT_FLYE10_NIFRLiF61.sorted.bam

## index bam ##
samtools index ${WKDIR}/bamfiles/BeeT_FLYE10_NIFRLiF61.sorted.bam


EOF



Submitted batch job 1404424


In [5]:
cd ${WKDIR}/bamfiles/
mkdir sorted
mv *.sorted.bam* sorted
rm -rf *am
mv sorted/* .
rm -rf sorted/

In [8]:
# Create Blast files.


sbatch << EOF
#!/bin/bash
#SBATCH -J megablast
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=32
#SBATCH --mem=10G
#SBATCH --time=11:00:00

module load ncbi-blast/2.11.0

#Flye10
blastn \
-task megablast \
-query ${FLYE10} \
-db /workspace/ComparativeDataSources/NCBI/nt/nt \
-outfmt '6 qseqid staxids bitscore std' \
-max_target_seqs 1 \
-max_hsps 1 \
-num_threads 32 \
-evalue 1e-25 \
-out ${WKDIR}/blasthits/FLYE10.vs.nt.mts1.hsp1.1e25.megablast.out




EOF



Submitted batch job 1404123


In [3]:
sbatch << EOF
#!/bin/bash
#SBATCH -J blobtools
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=2
#SBATCH --mem=1G
#SBATCH --time=2:00:00

module load conda
conda activate hraijc_blobtools
/workspace/hraijc/git_clones/blobtools/blobtools --version

/workspace/hraijc/git_clones/blobtools/blobtools create \
 -i ${FLYE10} \
 -b ${WKDIR}/bamfiles/BeeT_FLYE10_ONT15.sorted.bam \
 -t ${WKDIR}/blasthits/FLYE10.vs.nt.mts1.hsp1.1e25.megablast.out \
 -o ${WKDIR}/FLYE10.blobplot

 
 
/workspace/hraijc/git_clones/blobtools/blobtools plot \
 -i ${WKDIR}/FLYE10.blobplot.blobDB.json \
 -o ${WKDIR}/FLYE10.blobplot
 
 
 
/workspace/hraijc/git_clones/blobtools/blobtools view \
 -i ${WKDIR}/FLYE10.blobplot.blobDB.json \
 -o ${WKDIR}/FLYE10.blobplot


EOF

Submitted batch job 1404427


In [2]:
#Plot popgen sample coverage.

sbatch << EOF
#!/bin/bash
#SBATCH -J blobtools
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err
#SBATCH --cpus-per-task=2
#SBATCH --mem=1G
#SBATCH --time=1:00:00


module load conda
conda activate hraijc_blobtools

/workspace/hraijc/git_clones/blobtools/blobtools create \
 -i ${FLYE10} \
 -b ${WKDIR}/bamfiles/BeeT_FLYE10_NIFRLiF61.sorted.bam \
 -t ${WKDIR}/blasthits/FLYE10.vs.nt.mts1.hsp1.1e25.megablast.out \
 -o ${WKDIR}/BeeT_FLYE10_NIFRLiF61.blobplot

/workspace/hraijc/git_clones/blobtools/blobtools plot \
 -i ${WKDIR}/BeeT_FLYE10_NIFRLiF61.blobplot.blobDB.json \
 -o ${WKDIR}/BeeT_FLYE10_NIFRLiF61.blobplot

/workspace/hraijc/git_clones/blobtools/blobtools view \
 -i ${WKDIR}/BeeT_FLYE10_NIFRLiF61.blobplot.blobDB.json \
 -o ${WKDIR}/BeeT_FLYE10_NIFRLiF61.blobplot
 
EOF

Submitted batch job 1404425


In [1]:
# CP blobplot to github img folder.
cp /workspace/hraijc/Bee/BeeT/Assemblies/blobplots/BeeT_FLYE10_NIFRLiF61.blobplot.BeeT_FLYE10_NIFRLiF61.blobplot.blobDB.json.bestsum.phylum.p8.span.100.blobplot.bam0.png /workspace/hraijc/Gitrepos/High-quality-genomes/Leioproctus/img/BeeT_FLYE10_NIFRLiF61.blobplot.png